In [ ]:

import os, sys, numpy as np, torch
sys.path.append("..")
from src.config import ModelParams, TrainConfig
from src.deqn import PolicyNetwork, Trainer
from src.steady_states import solve_flexprice_sss, export_rbar_tensor, commitment_init_from_sss, solve_taylor_sss, solve_commitment_sss_switching, solve_discretion_sss_switching


In [ ]:

params = ModelParams(device="cpu", dtype=torch.float32)
cfg = TrainConfig(steps=2, batch_size=32)
flex = solve_flexprice_sss(params)
rbar = export_rbar_tensor(params, flex)
commit_init = commitment_init_from_sss(params)

print("Flex-price:", flex.by_regime)
print("rbar:", rbar)
print("commit_init:", commit_init)


taylor_sss = solve_taylor_sss(params, flex)
commit_sss = solve_commitment_sss_switching(params)
disc_sss = solve_discretion_sss_switching(params)
print('Taylor SSS:', taylor_sss.by_regime)
print('Commitment SSS:', commit_sss.by_regime)

print("Taylor SSS:", taylor_sss)
print("Commitment SSS:", commit_sss.by_regime)
print("Discretion SSS:", disc_sss.by_regime)


In [ ]:

def smoke(policy):
    if policy in ["taylor","mod_taylor"]:
        d_in, d_out = 5, 8
    elif policy=="discretion":
        d_in, d_out = 5, 11
    elif policy=="commitment":
        d_in, d_out = 7, 13
    net = PolicyNetwork(d_in, d_out, hidden=cfg.hidden_layers, activation=cfg.activation)
    tr = Trainer(params=params, cfg=cfg, policy=policy, net=net, rbar_by_regime=rbar if policy=="mod_taylor" else None)
    x0 = tr.simulate_initial_state(64, commitment_sss=commit_init if policy=="commitment" else None)
    resid = tr._residuals(x0)
    assert resid.shape[-1] == len(tr.res_keys)
    assert torch.isfinite(resid).all()
    return float((resid**2).mean())

for pol in ["taylor","mod_taylor","discretion","commitment"]:
    print(pol, smoke(pol))
print("✅ Smoke tests passed.")
